<a href="https://colab.research.google.com/github/riccardomarin/Tutorial_Spectral_Geometry/blob/main/inverse/01_Isospectralization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will see tha application of some computational inverse geometry techniques to a couple of trivial examples.

We want to solve the general optimization problem of:

\begin{align}
\arg\min _{\mathbf{X} \in \mathbb{R}^{n \times d}}\left\|\boldsymbol{\lambda}\left(\boldsymbol{\Delta}(\mathbf{X})\right)-\boldsymbol{\mu}\right\|_{\omega}+\rho_{X}(\mathbf{X})
\end{align}

References:
* Cosmo, Luca, et al. "Isospectralization, or how to hear shape, style, and correspondence." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019.

Related repositories:
* https://github.com/lcosmo/isospectralization
* https://github.com/AriannaRampini/InverseSpectralGeometry_3DVTutorial

In [ ]:
!git clone https://github.com/riccardomarin/Tutorial_Spectral_Geometry.git
%cd Tutorial_Spectral_Geometry/inverse

!pip install plyfile

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
DEVICE = 'cpu'

In [ ]:
import sys
sys.path.append('../utils')
    
import numpy as np
import torch

import utils_mesh 
from utils_spectral import LB_FEM_sparse as lbo, EigendecompositionSparse as eigh

Few considerations on computing the gradient of the laplacian:
* if we keep the connectivity of the mesh fixed, the LBO operator is differentiable with respect to the point positions. It is indeed expressed in terms of the lenght (or angles) of each edge, which is computed from the 3d coordinates of each point.

* Eigenvalues of any matrix A are also differentiable with respect to the entries of the matrix

* We can thus exploit autodifferntiation capabilities of torch (or the autodiff package you may prefer) to automatically compute the eigenvalues w.r.t. the points coordinates of the input mesh.

Let's start with a trivial example...

In [ ]:
triangle_1 = torch.tensor([[0,0,0],[0,1,0],[1,0,0]]).double()
triangle_2 = torch.tensor([[0,0,0],[0,2,0],[1,0,0]]).double()
tri = torch.tensor([[0,1,2]])

utils_mesh.plot_colormap([triangle_1,triangle_2],[tri]*2,[None]*2)

We can compute and visualize the first eigenvectors and eigenvalues of the two tringular meshes

In [ ]:
from utils_spectral import LB_cotan as lbo, Eigendecomposition as eigh

stiff, lumped_mass = lbo(triangle_1,tri)

#we have to convert the generalized eigendecomposition problem to the simpler eigendecomposition of hermitian matrices
inv_sqrt_mass = lumped_mass.rsqrt()
L1 = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
evecs_1,evals_1 = eigh(L1,3)
evecs_1 = evecs_1*inv_sqrt_mass[:,None]

stiff, lumped_mass = lbo(triangle_2,tri)
inv_sqrt_mass = lumped_mass.rsqrt()
L2 = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
evecs_2,evals_2 = eigh(L2,3)
evecs_2 = evecs_2*inv_sqrt_mass[:,None]

fig = utils_mesh.plot_colormap([triangle_1]*3,[tri]*3,[e for e in evecs_1])
fig.show()

fig = utils_mesh.plot_colormap([triangle_2]*3,[tri]*3,[e for e in evecs_2])
fig.show()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(evals_1.data.cpu())
plt.plot(evals_2.data.cpu())
plt.show()


### Can we deform triangle_2 into trinagle_1 just knowing the eigenvalues of traingle_1?
Let's write down it as an optimization problem

In [ ]:
target_evals = evals_1.detach().clone().to(DEVICE)

X = torch.nn.Parameter(triangle_2.detach().clone().to(DEVICE)) # we start from the original vertex coordinates of triangle_2
tri = tri.to(DEVICE)

optimizer = torch.optim.Adam(params=[X],lr=5e-3)

for i in range(1000): #we now enter the iterative optimization loop
    optimizer.zero_grad() #usual pytorch code for gradient descent
    
    stiff, lumped_mass = lbo(X,tri)
    inv_sqrt_mass = lumped_mass.rsqrt()
    Lopt = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
    _,evals_opt = eigh(Lopt,3)
    
    loss = torch.sum((evals_opt-target_evals)[1:]**2) #the first eigenvalue will always be almost 0
    
    loss.backward() #let's compute the gradient w.r.t. optimiziation parameters
    
#     print(X.grad)
    if i %10==0:
        print('Loss: %.2e' % loss)
    
    optimizer.step()#usual pytorch code for gradient descent

In [ ]:
import matplotlib.pyplot as plt
plt.plot(evals_1.data.cpu(),'o')
plt.plot(evals_opt.data.cpu(),'r')
plt.plot(evals_2.data.cpu(),'b')
plt.show()


In [ ]:
fig = utils_mesh.plot_colormap([triangle_2, X.data.cpu(), triangle_1],[tri]*3,[None]*3)
fig.show()

As we already discussed, in the general case isospectral != isometric.

Moreover, you may have noticed that we are using just the first 3 eigenvalues of the triangle (we are limited by the first order FEM). Since the first eigenvalue is always null, we can fix just just 2 dof, while a generic triangle has 3.

But what if we have some prior domain knowlege on the domain of the shape that we want to reconstruct?

Assume, for instance, that we are interested in (almost) orthogonal triangles.
we can encode this information as a regularizer (possibly an hard constraint) in the optimization problem:

we want (x[2]-X[0]) to be orthogonal to (X[1]-X[0])

In [ ]:
target_evals = evals_1.detach().clone().to(DEVICE)

X = torch.nn.Parameter(triangle_2.detach().clone().to(DEVICE)) # we start from the original vertex coordinates of triangle_2
tri = tri.to(DEVICE)

optimizer = torch.optim.Adam(params=[X],lr=1e-2)

import time
t=time.time()
for i in range(2000): #we now enter the iterative optimization loop
    optimizer.zero_grad() #usual pytorch code for gradient descent
    
    stiff, lumped_mass = lbo(X,tri)
    inv_sqrt_mass = lumped_mass.rsqrt()
    Lopt = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
    _,evals_opt = eigh(Lopt,3)
    
    loss_eig = torch.sum((evals_opt-target_evals)[1:]**2) #the first eigenvalue will always be almost 0
    loss_ortho = 1e2*torch.dot(X[2]-X[0], X[1]-X[0])**2
    loss = loss_eig+loss_ortho
    
    loss.backward() #let's compute the gradient w.r.t. optimiziation parameters

    optimizer.step()#usual pytorch code for gradient descent
    if i %10==0:
        print('Loss: %.2e' % loss)    
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(evals_1.data.cpu(),'o')
plt.plot(evals_opt.data.cpu(),'r')
plt.plot(evals_2.data.cpu(),'b')
plt.show()


In [ ]:
fig = utils_mesh.plot_colormap([triangle_2, X.data.cpu(), triangle_1],[tri]*3,[None]*3)
fig.show()

Attention: Rigid transformations are isoemtries.
Have a look to the metric of the optimized triangle:

In [ ]:
#Let's check edge lengths:
print(torch.norm(X[0]-X[1]))
print(torch.norm(X[1]-X[2]))
print(torch.norm(X[2]-X[0]))


### A more interesting (and harder) example

What about "general" planar shapes? 

In [ ]:
VERT1,TRIV1 = utils_mesh.load_ply('../data/mickey.ply')
VERT2,TRIV2 = utils_mesh.load_ply('../data/oval.ply')

VERT1 = torch.tensor(VERT1).double().to(DEVICE)
VERT2 = torch.tensor(VERT2).double().to(DEVICE)
TRIV1 = torch.tensor(TRIV1).long().to(DEVICE)
TRIV2 = torch.tensor(TRIV2).long().to(DEVICE)

fig = utils_mesh.plot_colormap([VERT1,VERT2],[TRIV1, TRIV2],[None]*2)
fig.show()

In [ ]:
from utils_spectral import LB_cotan as lbo, Eigendecomposition as eigh

#how many eigenvalues do we wish to align? Higher eigenvalues are usually dominated by discretization noise
k = 20

stiff, lumped_mass = lbo(VERT1,TRIV1)
inv_sqrt_mass = lumped_mass.rsqrt()
L1 = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
evecs_1,evals_1 = eigh(L1,k)

stiff, lumped_mass = lbo(VERT2,TRIV2)
inv_sqrt_mass = lumped_mass.rsqrt()
L2 = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
evecs_2,evals_2 = eigh(L2,k)

#extract the vertex indexes for each edge of the mesh, it will be used later to compute edges' length 
edge_indexes2 = np.nonzero(stiff.data.cpu().triu(1))

There are some problems we need to take care of in the optimization process:
* Triangle flips should not be allowed
* Very skewed triangles cause numerical instability (especially if using first order Laplacian approximation)

In [ ]:
import torch.linalg
from torch.nn.functional import normalize
target_evals = evals_1.detach().clone()

X = torch.nn.Parameter(VERT2.detach().clone().to(DEVICE)) # we start from the original vertex coordinates of triangle_2
tri = TRIV2.to(DEVICE)

optimizer = torch.optim.Adam(params=[X],lr=5e-3)

for i in range(1000): #we now enter the iterative optimization loop
    optimizer.zero_grad() #usual pytorch code for gradient descent
    
    stiff, lumped_mass = lbo(X,tri)
    inv_sqrt_mass = lumped_mass.rsqrt()
    Lopt = inv_sqrt_mass[:,None]*stiff*inv_sqrt_mass[None,:]
    _,evals_opt = eigh(Lopt,k)

    
    loss = torch.sum((evals_opt-target_evals)[1:]**2/torch.arange(1,k).to(X.device)) #the first eigenvalue will always be almost 0
    
    #regularizers
    
    #triangle flips
    tripts = X[tri]
    loss_flip =  (torch.cross((-tripts[:,1,:]+tripts[:,0,:]),(tripts[:,2,:]-tripts[:,0,:]),dim=1)[:,-1]+1e-3).relu().pow(2).sum()
    loss_flip += (torch.cross((-tripts[:,2,:]+tripts[:,1,:]),(tripts[:,0,:]-tripts[:,1,:]),dim=1)[:,-1]+1e-3).relu().pow(2).sum()
    loss_flip += (torch.cross((-tripts[:,0,:]+tripts[:,2,:]),(tripts[:,1,:]-tripts[:,2,:]),dim=1)[:,-1]+1e-3).relu().pow(2).sum()
    loss = loss + 1e5*loss_flip
    
    #edge length
    loss_len = (X[edge_indexes2[:,0],:]-X[edge_indexes2[:,1],:]).pow(2).sum(-1).mean()
    loss = loss  + 1e1*loss_len
                         
    
    loss.backward() #let's compute the gradient w.r.t. optimiziation parameters
    torch.nn.utils.clip_grad_norm_([X], 1e-3)
#     print(X.grad)
    if i %10==0:
        print('Loss: %.2e' % loss)
#         fig = utils_mesh.plot_colormap([ X.data.cpu()],[TRIV2]*3,[None]*3,wireframe=True)
#         fig.show()
#         print(time.time()-t)
#         t=time.time()
    optimizer.step()#usual pytorch code for gradient descent

The initial triangulation is not always ideal for representing the target shape. Resampling the optimized shape every once in a while would help the optimization process.

In [ ]:
# X = torch.load('results/mickey_opt.pt',map_location=torch.device(DEVICE))

fig = utils_mesh.plot_colormap([ VERT1, X.data.cpu()],[TRIV1,TRIV2],[None]*2,wireframe=True)
fig.show()

## 3D Shapes

In [ ]:
VERT1,TRIV1 = utils_mesh.load_ply('../data/round_cuber_out.ply')
VERT2,TRIV2 = utils_mesh.load_ply('../data/round_cuber.ply')

VERT1 = torch.tensor(VERT1).double().to(DEVICE)
VERT2 = torch.tensor(VERT2).double().to(DEVICE)
TRIV1 = torch.tensor(TRIV1).long().to(DEVICE)
TRIV2 = torch.tensor(TRIV2).long().to(DEVICE)

fig = utils_mesh.plot_colormap([VERT1,VERT2],[TRIV1, TRIV2],[None]*2)
fig.show()

In [ ]:
k = 30

stiff_1, lumped_mass_1 = lbo(VERT1,TRIV1)
inv_sqrt_mass_1 = lumped_mass_1.rsqrt()
L1 = inv_sqrt_mass_1[:,None]*stiff_1*inv_sqrt_mass_1[None,:]
evecs_1,evals_1 = eigh(L1,k)

stiff_2, lumped_mass_2 = lbo(VERT2,TRIV2)
inv_sqrt_mass_2 = lumped_mass_2.rsqrt()
L2 = inv_sqrt_mass_2[:,None]*stiff_2*inv_sqrt_mass_2[None,:]
evecs_2,evals_2 = eigh(L2,k)

###  Challenges of 3D shapes:
* Even if triangle flips are not in general possible in a 3D embedding, ugly triangles and spikes are still a problem. We can alleviate this problem adding a smoothness prior.
* In 3D shapes there exist many "non-meaningful" isometries. For instance, flipping any protuberation inside out is a valid isometry. 

In [ ]:
import torch.linalg
from torch.nn.functional import normalize
target_evals = evals_1.detach().clone()

iX = VERT2.detach().clone().to(DEVICE)
dX = torch.nn.Parameter((VERT2*0).detach().clone().to(DEVICE)) # we start from the original vertex coordinates of triangle_2
tri = TRIV2.to(DEVICE)

optimizer = torch.optim.Adam(params=[dX],lr=1e-3)

for i in range(5000): #we now enter the iterative optimization loop
    optimizer.zero_grad() #usual pytorch code for gradient descent
 
    X = iX+dX
    stiff_o, lumped_mass_o = lbo(X,tri)
    inv_sqrt_mass_o = lumped_mass_o.rsqrt()
    Lopt = inv_sqrt_mass_o[:,None]*stiff_o*inv_sqrt_mass_o[None,:]
    _,evals_opt = eigh(Lopt,k)

    
    loss = torch.sum((evals_opt-target_evals)[1:]**2/torch.arange(1,k).to(X.device)) #the first eigenvalue will always be almost 0
    
    ## regularizers 
    
    #curvature
    loss_curv = 1e1*torch.norm((stiff_o)@dX)
    loss = loss + loss_curv

    #volume
    tripts = X[tri]
    cross =  torch.cross((tripts[:,1,:]-tripts[:,0,:]),(tripts[:,1,:]-tripts[:,2,:]),dim=1)
    volume = 2e0*torch.sum(tripts.sum(1)*cross)
    loss = loss-volume
        
    
    loss.backward() #let's compute the gradient w.r.t. optimiziation parameters
    torch.nn.utils.clip_grad_norm_([X], 1e-3)
#     print(X.grad)
    if i %10==0:
        print('Loss: %.2e (%.2e)' % (loss,loss_curv))
#         fig = utils_mesh.plot_colormap([ X.data.cpu()],[TRIV2]*3,[None]*3,wireframe=True)
#         fig.show()
#         print(time.time()-t)
#         t=time.time()
    optimizer.step()#usual pytorch code for gradient descent

In [ ]:
# X = torch.load('results/cube_noreg.pt',map_location=torch.device(DEVICE))
X = torch.load('results/cube_res_all.pt',map_location=torch.device(DEVICE))
# X = torch.load('results/cube_res_in.pt',map_location=torch.device(DEVICE))

stiff_o, lumped_mass_o = lbo(X,tri)
inv_sqrt_mass_o = lumped_mass_o.rsqrt()
Lopt = inv_sqrt_mass_o[:,None]*stiff_o*inv_sqrt_mass_o[None,:]
evecs_opt,evals_opt = eigh(Lopt,k)
err = torch.norm(((inv_sqrt_mass_o[:,None]**2)*stiff_o)@dX,dim=-1)

fig = utils_mesh.plot_colormap([ VERT1, X.data.cpu()],[TRIV1,TRIV2],[None,err],wireframe=True)
fig.show()

plt.plot(target_evals.detach().cpu())
plt.plot(evals_opt.detach().cpu())
plt.show()


plt.plot(target_evals.detach().cpu()-evals_opt.detach().cpu())

Also eigenvectors are now more similar

In [ ]:
evecs_opt,evals_opt = eigh(Lopt,k)
evecs_opt = evecs_opt * inv_sqrt_mass_o[:,None]

evecs_1,evals_1 = eigh(L1,k)
evecs_1 = evecs_1 * inv_sqrt_mass_1[:,None]

evecs_2,evals_2 = eigh(L2,k)
evecs_2 = evecs_2 * inv_sqrt_mass_2[:,None]

print('Original eigenvectors')
fig = utils_mesh.plot_colormap([VERT2]*3,[TRIV2]*3,[evecs_2[:,1],evecs_2[:,2],evecs_2[:,10]])
fig.show()

print('Optimized eigenvectors')
fig = utils_mesh.plot_colormap([X.data.cpu()]*3,[TRIV2]*3,[evecs_opt[:,1],evecs_opt[:,2],evecs_opt[:,10]])
fig.show()

print('Target eigenvectors')
fig = utils_mesh.plot_colormap([VERT1]*3,[TRIV1]*3,[evecs_1[:,1],evecs_1[:,2],evecs_1[:,10]])
fig.show()


This affects also the quality of the functional map, and can be used as a preconditioning step of the shapes to be matched.

In [ ]:
# CA=B
# CA=psi*phi'

C_before = (evecs_1.t()*lumped_mass_1[None,:])@evecs_2
C_after  = (evecs_1.t()*lumped_mass_1[None,:])@evecs_opt


fig, ax = plt.subplots(1,2,figsize=(10,6))
ax[0].imshow(C_before.data.cpu().abs(),cmap=plt.get_cmap('Reds'))
ax[0].set_title('before')
ax[1].imshow(C_after.data.cpu().abs(),cmap=plt.get_cmap('Reds'))
ax[1].set_title('after')

plt.show()